# Data Validation with TensorFlow Data Validation (TFDV)
Data validation is the process of examining data to ensure that they can be properly input before training (training data) or inference (serving data).
One way to gain confidence in the validity of data is to prove conformance to a given formal specification.

[TensorFlow Data Validation](https://github.com/tensorflow/data-validation) (TFDV) is a library for exploring and validating machine learning data. It is designed to be highly scalable and to work well with TensorFlow and TensorFlow Extended (TFX). Below is a simple example of data validation with TFDV.

NB: This notebook's runtime type is Python 2.

## Set-up

In [1]:
!apt-get install python-dev python-snappy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-dev is already the newest version (2.7.15~rc1-1).
The following NEW packages will be installed:
  python-snappy
0 upgraded, 1 newly installed, 0 to remove and 28 not upgraded.
Need to get 10.8 kB of archives.
After this operation, 39.9 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-snappy amd64 0.5-1.1build2 [10.8 kB]
Fetched 10.8 kB in 0s (257 kB/s)
Selecting previously unselected package python-snappy.
(Reading database ... 132681 files and directories currently installed.)
Preparing to unpack .../python-snappy_0.5-1.1build2_amd64.deb ...
Unpacking python-snappy (0.5-1.1build2) ...
Setting up python-snappy (0.5-1.1build2) ...


In [2]:
!pip install -q tensorflow_data_validation

     |████████████████████████████████| 2.3MB 2.4MB/s 
     |████████████████████████████████| 225kB 60.4MB/s 
     |████████████████████████████████| 1.9MB 41.9MB/s 
     |████████████████████████████████| 2.9MB 39.6MB/s 
     |████████████████████████████████| 440kB 55.9MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 1.0MB 45.1MB/s 
     |████████████████████████████████| 81kB 9.6MB/s 
     |████████████████████████████████| 122kB 54.4MB/s 
     |████████████████████████████████| 143kB 57.3MB/s 
     |████████████████████████████████| 235kB 61.9MB/s 
     |████████████████████████████████| 92kB 10.6MB/s 
     |████████████████████████████████| 512kB 56.4MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
ERROR: tensorflow-serving-api 2.0.0 has requirement tensorflow~=2.0.0, but you'll have tensorflow 1.15.0 which is incompatible.
ERROR: apache-beam 2.16.0 has requirement 

In [3]:
import pandas as pd
import tensorflow as tf
import tensorflow_data_validation as tfdv

from __future__ import print_function

/usr/local/lib/python2.7/dist-packages/apache_beam/__init__.py:84: UserWarning: You are using Apache Beam with Python 2. New releases of Apache Beam will soon support Python 3 only.
  'You are using Apache Beam with Python 2. '


## Basic Dataset Description

In [5]:
dataset = pd.read_csv("pollution_small.csv")
dataset.shape

(2188, 5)

In [6]:
trainingdset = dataset[:1600]
trainingdset.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


In [7]:
testdset = dataset[1600:]
testdset.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


## Basic Data Analysis and Validation
[Get started with TFDV](https://www.tensorflow.org/tfx/data_validation/get_started)

### Generate Statistics and Visualize

In [8]:
traindset_stats = tfdv.generate_statistics_from_dataframe(dataframe=dataset)
tfdv.visualize_statistics(traindset_stats)

### Infering the schema

In [9]:
schema = tfdv.infer_schema(statistics=traindset_stats)
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'soot',FLOAT,required,,-
'no2',FLOAT,required,,-
'pm10',FLOAT,required,,-
'Date',BYTES,required,,-
'so2',FLOAT,required,,-


## Test Dataset Validation
Does the test dataset conform to the Schema?

### Check for Anomalies
The original test dataset we are using does not differ from the data specification.

In [10]:
testdset_stats = tfdv.generate_statistics_from_dataframe(dataframe=testdset)
anomalies = tfdv.validate_statistics(statistics=testdset_stats, schema=schema)
tfdv.display_anomalies(anomalies)

However, a "corrupted" version of the original test dataset will not comply with the specification.

In [14]:
testdset_copy = testdset.copy()
testdset_copy["pm10"].replace(6.38,"six point thirty eight",inplace=True)
testdset_copy.drop("soot", axis=1, inplace=True)
testdset_copy_stats = tfdv.generate_statistics_from_dataframe(dataframe=testdset_copy)
anomalies_testdset_copy = tfdv.validate_statistics(statistics=testdset_copy_stats, schema=schema)
tfdv.display_anomalies(anomalies_testdset_copy)

,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


##  Dealing with Schemas in Production
Implementing machine learning (and in particular deep learning) applications at production level requires to be cognizant of the end-to-end pipeline.

Under the above consideration, it is advisable to define a single schema for all of the data that could ever flow all along the pipeline, but at the same time it is advisable to define subsets of the original schema (referred to as environments) that define subsets of the data that could flow at particular stages of the pipeline.

In [0]:
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

tfdv.get_feature(schema, "soot").not_in_environment.append("SERVING")

### Checking for anomalies between the SERVING environment and new test set
Checking for anomalies on data expected in the inference phase of the ML application should be performed relative to the SERVING environment. In our running example, the ommision of column `soot` should not yield an alert.

In [16]:
anomalies_env_serving = tfdv.validate_statistics(testdset_copy_stats, schema, environment="SERVING")
tfdv.display_anomalies(anomalies_env_serving)